# Day 12: Garden Groups

In [1]:
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt


def read_data(is_test: bool = False):
    if is_test:
        with open('data/2024-12-example.txt', 'r') as f:
            data = f.readlines()
    else:
        with open('data/2024-12.txt', 'r') as f:
            data = f.readlines()
    data = [x.strip() for x in data]
    return data

In [2]:
def label_regions(str_data: list, plot: bool = False):
    # convert input data to array    
    plant_types = list(set(''.join(str_data)))
    arr = np.zeros((len(str_data), len(str_data[0])))
    for i in range(len(str_data)):
        for j in range(len(str_data[i].strip())):
            arr[i, j] = plant_types.index(str_data[i][j]) + 1
    
    if plot:
        plt.imshow(arr)
        plt.axis('off')
        plt.title('Areas')
        plt.show()

    return arr


def label_areas(region_array, plot: bool = False):
    # label the regions
    label_array = measure.label(region_array.astype(int), connectivity=1)

    if plot:
        plt.imshow(label_array)
        plt.title('Labeled Areas')
        plt.axis('off')
        plt.show()
    return label_array


def calculate_perimeter(label_array, plot: bool = False):
    # start with all perimeters as 4
    perimeters = np.ones_like(label_array) * 4
    
    # remove count for each adjacent pixel with the same label
    for i in range(label_array.shape[0]):
        for j in range(label_array.shape[1]):
            c_label = label_array[i, j]
            # left side
            if i > 0:
                if c_label == label_array[i-1, j]:
                    perimeters[i, j] -= 1
            # right side
            if i < label_array.shape[0] - 1:
                if c_label == label_array[i+1, j]:
                    perimeters[i, j] -= 1
            # top side
            if j > 0:
                if c_label == label_array[i, j-1]:
                    perimeters[i, j] -= 1
            # bottom side
            if j < label_array.shape[1] - 1:
                if c_label == label_array[i, j+1]:
                    perimeters[i, j] -= 1

    
    if plot:
        plt.imshow(perimeters)
        plt.title('Perimeters')
        plt.axis('off')
        plt.show()
    
    return perimeters


def compute_price_perimeter(label_array, perimeter):
    total_price = 0
    for region in measure.regionprops(label_array):
        # compute perimeter of region
        region_perimeter = 0
        for c in region.coords:
            region_perimeter += perimeter[c[0], c[1]]

        # compute cost of region
        total_price += region_perimeter * int(region.area)
    return total_price

## Part 1

In [35]:
data = read_data(is_test=False)
region_array = label_regions(data, plot=False)
labels = label_areas(region_array, plot=False)
perimeters = calculate_perimeter(labels, plot=False)
print('Part 1:', compute_price_perimeter(labels, perimeters))

Part 1: 1424472


## Part 2

In [33]:
def calculate_price_sides(label_array, perimeter):
    total_price = 0
    for region in measure.regionprops(label_array):
        # compute number of sides in region
        region_sides = 0
        for c in region.coords:
            region_perimeter += perimeter[c[0], c[1]]

        # compute cost of region
        total_price += region_perimeter * int(region.area)
    return total_price


def coord_exists(region_coords, coord):
    return (coord == region_coords).all(axis=1).any().item()


def increase_count(region_coords, c, c1, c2) -> int:
    if coord_exists(region_coords, c) or (coord_exists(region_coords, c1) and not coord_exists(region_coords, c2)):
        return 0
    return 1

In [36]:
price = 0
for region in measure.regionprops(labels):
     region_perimeter = 0
     coords = region.coords
     for coord in coords:
          # compute all directions
          north = np.array(coord) + np.array([-1, 0])
          west =  np.array(coord) + np.array([0, -1])
          south = np.array(coord) + np.array([1, 0])
          east = np.array(coord) + np.array([0, 1])
          
          nw = np.array(coord) + np.array([-1, -1])
          sw = np.array(coord) + np.array([1, -1])
          ne = np.array(coord) + np.array([-1, 1])

          # top is an edge
          region_perimeter += increase_count(coords, north, west, nw)
          # bottom is an edge
          region_perimeter += increase_count(coords, south, west, sw)
          # left is an edge
          region_perimeter += increase_count(coords, west, north, nw)
          # right is an edge
          region_perimeter += increase_count(coords, east, north, ne)
     price += region_perimeter * int(region.area)

print('Part 2:', price)

Part 2: 870202
